# Griffe 入门教程

`griffe` 是一个强大的 Python 库，用于从源代码中静态地提取文档字符串、类型注解、函数签名等信息。它对于自动化生成 API 文档、代码分析和静态检查非常有用。

这个 Notebook 将带你了解 `griffe` 的基本用法。

首先，我们需要安装 `griffe`：

In [ ]:
!pip install griffe

### 1. 加载一个模块

`griffe` 的核心功能是加载一个模块并将其代码结构解析为一个可供程序访问的对象。我们使用 `griffe.loader.load` 函数来完成这个任务。

我们将加载与此 Notebook 位于同一目录下的 `my_module.py` 文件。

In [9]:
import griffe

# 加载我们创建的 my_module.py 模块
module_data = griffe.load("my_module")

### 2. 探索模块对象

加载成功后，`module_data` 就成了一个代表 `my_module` 的 `Module` 对象。我们可以通过它访问模块的所有信息。

In [10]:
print(f"模块名称: {module_data.name}")
print(f"模块路径: {module_data.filepath}")
print("模块文档字符串:")
print(module_data.docstring.value)

模块名称: my_module
模块路径: D:\Awesome-demo\Small_but_Power_Python_Library\my_module.py
模块文档字符串:
This is a sample module demonstrating various Python elements.


### 3. 访问模块成员

`Module` 对象有一个 `members` 属性，它是一个字典，包含了模块中定义的所有成员（类、函数等）。

In [11]:
print("模块成员:")
for member_name, member_object in module_data.members.items():
    print(f"- {member_name} (类型: {member_object.kind.value})")

模块成员:
- MODULE_VERSION (类型: attribute)
- greet (类型: function)
- Person (类型: class)
- _private_function (类型: function)
- _PrivateClass (类型: class)


你会注意到，`_private_function` 这个以下划线开头的“私有”函数也被加载了。`griffe` 默认会加载所有成员。我们可以稍后看到如何过滤它们。

### 4. 深入了解函数对象

我们可以从 `members` 字典中获取一个特定的函数对象，并检查它的详细信息，比如参数和返回类型。

In [ ]:
simple_func = module_data["greet"]

print(f"函数名称: {simple_func.name}")
print("文档字符串:")
print(simple_func.docstring.value)

print("参数详情:")
for param in simple_func.parameters:
    print(f"- 名称: {param.name}, 注解: {param.annotation}, 默认值: {param.default}")

print(f"返回类型注解: {simple_func.returns}")

函数名称: greet
文档字符串:
Greet the given name.

Args:
    name: The name to greet.
    
Returns:
    A greeting message.
参数详情:
- 名称: name, 注解: str, 默认值: None
返回类型注解: str


### 5. 深入了解类对象

同样，我们也可以检查类及其成员。

In [16]:
my_class = module_data["Person"]

print(f"类的名称: {my_class.name}")
print("类的文档字符串:")
print(my_class.docstring.value)

print("类的成员:")
for member_name, member_object in my_class.members.items():
    # 我们只关心方法，可以过滤掉属性等其他成员
    if member_object.kind.value == 'function':
        print(f"- 方法: {member_name}")

类的名称: Person
类的文档字符串:
A class representing a person.
类的成员:
- 方法: __init__
- 方法: introduce


### 6. 过滤成员

在实际应用中，我们通常不希望文档包含私有成员（如 `_internal_function`）。`griffe` 允许我们在加载时提供搜索路径和过滤选项。

例如，我们可以通过 `docstring_filters` 过滤掉所有包含 `"internal"` 字符串的文档字符串，或者通过 `member_filters` 按名称过滤。

In [ ]:
import griffe

# 加载我们创建的 my_module.py 模块
module_data_filtered = griffe.load("my_module")

# 默认情况下，Griffe 会自动排除私有成员
public_members = [m for m in module_data_filtered.members if not m.startswith('_')]

print("过滤后的公共成员:")
for member_name in public_members:
    print(f"- {member_name}")

过滤后的公共成员:
- MODULE_VERSION
- greet
- Person


### 总结

`griffe` 是一个非常强大的工具，它让以编程方式访问 Python 代码的结构和文档变得简单。

主要优点包括：
- **静态分析**：无需执行代码即可获取信息，安全可靠。
- **信息全面**：可以获取函数签名、类型注解、文档字符串（并能解析）、装饰器等。
- **可扩展性**：支持自定义加载器和过滤器，以满足特定需求。

它是 `mkdocs-material` 的 `mkdocstrings` 插件的后端，是现代 Python 文档生成工具链中的关键一环。